In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
df = pd.read_json("/content/drive/MyDrive/JOKER/joker-2024-task2-classification-train (1).json")
df_test=pd.read_json("/content/drive/MyDrive/JOKER/joker-2024-task2-classification-test.json")
!pip install nltk


In [ ]:
import tensorflow as tf
from transformers import BertTokenizer, TFBertForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
# Assuming df is your DataFrame with 'text' and 'class' columns
X = df['text'].values
y = df['class'].values
# Encode the target variable
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)
# Load the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# Tokenize the input text
train_encodings = tokenizer(X_train.tolist(), truncation=True, padding=True)
test_encodings = tokenizer(X_test.tolist(), truncation=True, padding=True)
# Create TensorFlow datasets for training and testing
train_dataset = tf.data.Dataset.from_tensor_slices((
dict(train_encodings),
y_train
)).shuffle(len(X_train)).batch(8)
test_dataset = tf.data.Dataset.from_tensor_slices((
dict(test_encodings),
y_test
)).batch(8)
# Load the BERT model for sequence classification
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(label_encoder.classes_))
# Compile the model with sparse categorical crossentropy loss
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5),
loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
metrics=['accuracy'])
# Train the model
model.fit(train_dataset, epochs=3)
# Evaluate the model
_, accuracy = model.evaluate(test_dataset)
print("Accuracy:", accuracy)
# Make predictions
predictions = model.predict(test_dataset)
predicted_labels = label_encoder.inverse_transform(tf.argmax(predictions.logits, axis=1))
# Generate classification report
print(classification_report(label_encoder.inverse_transform(y_test), predicted_labels))
